# Graffiti

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
# example of using a pre-trained model as a classifier
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications.vgg16 import decode_predictions
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import VGG16
import os, numpy as np, pandas as pd, matplotlib.pyplot as plt
from subprocess import check_output
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [ ]:
X = np.load('/content/drive/MyDrive/final/xvgg.npy')
Y = np.load('/content/drive/MyDrive/final/yvgg.npy')


In [ ]:
from tensorflow.keras.layers import Flatten, Dense, Dropout

# Load VGG16 as a feature extractor (excluding the top classification layers)
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(400, 400, 3))

# Freeze the first few layers of the model
for layer in base_model.layers[:15]:
    layer.trainable = False

# Create new top layers for classification on the four classes
x = Flatten()(base_model.output)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)  # Dropout for regularization
x = Dense(4, activation='softmax')(x)  # Four classes

In [ ]:
from tensorflow.keras.models import Model

# Combine the base model and the new layers
model = Model(inputs=base_model.input, outputs=x)
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'], run_eagerly=True)

# View the structure of the model
model.summary()


Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 400, 400, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 400, 400, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 400, 400, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 200, 200, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 200, 200, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 200, 200, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 100, 100, 128)     0   

In [ ]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy, SparseCategoricalCrossentropy, CategoricalCrossentropy


for layer in model.layers[:-8]:
    layer.trainable = False


optimizer = Adam(learning_rate=0.001, beta_1=0.9)
model.compile(optimizer=optimizer, loss="CategoricalCrossentropy", metrics=['accuracy'])

In [ ]:
from sklearn.model_selection import KFold
from tensorflow import gather

train_losses = []
val_losses = []
train_accuracies = []
val_accuracies = []

kf = KFold(n_splits=5, shuffle=True, random_state=2)
for train_index, val_index in kf.split(x_train):
    X_train, X_val = gather(x_train, train_index), gather(x_train, val_index)
    Y_train, Y_val = gather(y_train, train_index), gather(y_train, val_index)

    # Train the model
    history = model.fit(X_train, Y_train, epochs=10, batch_size=32, validation_data=(X_val, Y_val))

    # Evaluate the model
    loss, accuracy = model.evaluate(X_val, Y_val)
    print(f'Validation Loss: {loss}, Validation Accuracy: {accuracy}\n')

    # store training and validation loss
    train_losses.extend(history.history['loss'])
    val_losses.extend(history.history['val_loss'])
    train_accuracies.extend(history.history['accuracy'])
    val_accuracies.extend(history.history['val_accuracy'])


In [ ]:
# epochs = len(train_losses) // 5  # assuming same number of epochs for each fold
epochs = 50
plt.figure(figsize=(12, 6))

# Plot losses
plt.subplot(1, 2, 1)
plt.plot(range(1, epochs+1), train_losses, label='Train Loss')
plt.plot(range(1, epochs+1), val_losses, label='Val Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Validation Losses')
plt.legend()

# Plot accuracies
plt.subplot(1, 2, 2)
plt.plot(range(1, epochs+1), train_accuracies, label='Train Accuracy')
plt.plot(range(1, epochs+1), val_accuracies, label='Val Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracies')
plt.legend()

plt.tight_layout()
plt.savefig('loss_accuracy_plot_vgg.png')
plt.show()

In [ ]:
model.save("/content/drive/MyDrive/final/kfoldvggfin.keras")
# from tensorflow.keras.models import load_model
# model = load_model('kfold.keras')

In [ ]:
y_pred = model.predict(x_test)
model.evaluate(x_test, y_test)